# Laphet (Version 0.7) with Myanmar Names Dataset
## Training with fasttext_freeze    

Shell script နဲ့ပဲ MLP, Bi-LSTM, Transformer, BERT, GPT အခြေခံတဲ့ language modelအကုန်ဆောက်ပြီး၊ name generation, testing အဆင့်ဆင့် လုပ်သွားပါမယ်။   

In [1]:
!pwd

/home/ye/exp/name-lm/lib


In [2]:
!cat ./train_test_name_ftfz.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 27 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/name/
mkdir -p output/name/
mkdir -p log/name/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/name/${model_type}.ftfz.model"
  local output_file="./output/name/${model_type}_ftfz_gen_texts.txt"
  local log_file="./log/name/${model_type}.ftfz.log"
  local train_data="./data/myRoman/train_name.txt"
  local dev_data="./data/myRoman/dev_name.txt"
  local test_data="./data/myRoman/test_name.txt"
  local start_name="./data/myRoman/start_names.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 \
      --lr 0.0001 --embedding_method fasttext_freeze \
      --fasttext_model ./fasttext

## Fasttext Model

ဒီ test run မှာ သုံးမယ့် fasttext embedding model က [Ngapi](https://github.com/ye-kyaw-thu/NgaPi) code တုန်းက သုံးခဲ့တဲ့ မော်ဒယ်နဲ့ အတူတူပါပဲ။ စာကြောင်းတွေကို syllable unit ဖြတ်ထားပြီး 100 dimension နဲ့ ဆောက်ထားတဲ့ မော်ဒယ်ပါ။  

Model download link:
[https://huggingface.co/ye-nlp/mySyllableFastText-Version1](https://huggingface.co/ye-nlp/mySyllableFastText-Version1)

myfasttext_v1.bin ဖိုင်ဆိုက်က ကြီးလို့ HuggingFace ရဲ့ personal account မှာပဲတင်ပေးထားလိုက်တယ်။

**embedding model က word နဲ့ ဖြတ်ထားရင် training, dev, test ဒေတာတွေကိုလည်း word unit နဲ့ ဖြတ်ပေးထားရပါမယ်။ fasttext embedding model ကို syllable unit နဲ့ ဖြတ်ဆောက်ထားရင် training, dev, test ဒေတာတွေကလည်း syllable ဖြတ်ထားမှ embedding မော်ဒယ်ကို apply လုပ်သွားမှာပါ။ ထိုနည်းလည်းကောင်း dimension command line argument ကိုလည်း ဆောက်ခဲ့တဲ့ မော်ဒယ်နဲ့ ညှိပေးထားရပါလိမ့်မယ်။   

In [14]:
!ls -lh ./fasttext-model/myfasttext*

-rw-rw-r-- 1 ye ye 766M Jan 27 16:37 ./fasttext-model/myfasttext_v1.bin
-rw-rw-r-- 1 ye ye 3.9M Jan 27 16:37 ./fasttext-model/myfasttext_v1.vec


training လုပ်တဲ့အခါမှာ .bin ဖိုင်ကို သုံးပါမယ်။  
.vec ဖိုင်ကတော့ text ဖိုင်မို့လို့ syllable တစ်လုံးချင်းစီရဲ့ vector information ကို ဖတ်လို့ရပါတယ်။  
အရင်ဆုံး ဖိုင်ရဲ့ ထိပ်ဆုံး စာကြောင်း ၁၀ကြောင်းကို လေ့လာကြည့်ရအောင်။  

In [15]:
!head ./fasttext-model/myfasttext_v1.vec

3462 100
အ -0.020812677 0.014222041 0.14707279 -0.1842381 -0.061052196 -0.057658885 -0.03292892 0.04936385 -0.025804132 0.0734657 -0.1159874 0.016686555 0.35442472 -0.14443621 0.05544194 0.12364729 -0.023075808 0.09519182 -0.045482293 0.029036311 -0.2193143 0.18958911 0.18115446 0.2333122 0.1195823 -0.008966078 0.411379 -0.08511002 0.047515854 -0.09509543 0.08582706 0.03881732 -0.055743393 0.03412059 -0.16639902 -0.28278458 -0.032576233 0.10848193 -0.020877324 -0.0009929277 -0.06336136 -0.14327583 -0.031124137 -0.08016338 -0.024092454 0.07601534 0.065982565 0.17565352 -0.10458869 0.18354851 0.1652404 0.09512043 -0.083493486 0.24238697 -0.17059515 0.010160673 -0.09070958 0.1116292 -0.037382837 0.2967968 -0.14358653 0.04922141 -0.22735184 0.01362576 -0.07403269 0.07475961 -0.19550946 -0.08912936 0.06503683 -0.14732634 -0.042020865 0.16236758 0.0344707 0.110176384 -0.04351531 0.02504056 -0.017584132 -0.019345423 0.14525683 0.018760469 -0.11468452 -0.16687019 0.12602636 -0.11606391 -0.0552

Vector ဖိုင်ရဲ့ နောက်ဆုံးပိုင်းကိုလည်း တချက် ကြည့်ကြည့်ရအောင်။  

In [16]:
!tail ./fasttext-model/myfasttext_v1.vec

လဗ္ဘေ 0.1830714 0.22337396 -0.21023625 -0.045918267 0.26395327 0.015270137 -0.27974412 -0.0021621864 0.36375126 0.25287792 -0.3485732 0.31900573 0.76218 -0.584555 0.2602841 0.2150892 0.015093825 0.40335682 -0.0021270078 -0.0102439625 -0.27154714 -0.24530803 -0.30234715 0.8159268 -0.33370203 0.16331537 -0.08953913 -0.15671305 0.073804915 0.21522263 0.37621176 -0.4310723 -0.07006551 0.19596152 0.26038486 -0.34378362 -0.29537085 0.2091432 0.28675535 0.13887826 0.22702347 -0.4010916 0.23250727 -0.37237856 0.3132686 -0.61562485 -0.258343 0.14362666 -0.036417037 -0.5316609 0.029325252 0.17908096 -0.42861366 0.21407056 -0.19330813 0.17727767 0.2880356 -0.028847774 -0.08432743 0.3579867 0.035279304 0.4984944 0.051906973 0.46864054 -0.03723403 -0.00057201437 0.34042454 0.23796026 -0.16938731 -0.42765442 0.023206659 0.11070449 0.43708295 -0.18668526 -0.28554085 0.1419076 -0.3099975 -0.31298527 -0.082222976 -0.35318598 0.26455504 -0.077361144 0.28766176 -0.2408238 -0.009982903 0.060138658 0.02775

Bash shell script ကို စ run ပြီ။  

In [3]:
!./train_test_name_ftfz.sh

Training Mlp language model:
Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 179.94it/s]
Epoch 1, Training Loss: 3.3194
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 522.86it/s]
Epoch 1, Validation Loss: 1.0696
Best model saved at ./model/name/mlp.ftfz.model with validation loss: 1.0696
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 194.29it/s]
Epoch 2, Training Loss: 1.0595
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 535.03it/s]
Epoch 2, Validation Loss: 1.0569
Best model saved at ./model/name/mlp.ftfz.model with validation loss: 1.0569
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 193.37it/s]
Epoch 3, Training Loss: 1.0536
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 517.16it/s]
Epoch 3, Validation Loss: 1.0549
Best model saved at ./model/name/mlp.ftfz.model with validation loss: 1.0549
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<

## Model, Vocab Information

In [6]:
!ls -lh ./model/name/*ftfz*model

-rw-rw-r-- 1 ye ye 3.4M Jan 29 00:25 ./model/name/bert.ftfz.model
-rw-rw-r-- 1 ye ye  88M Jan 29 00:23 ./model/name/bilstm.ftfz.model
-rw-rw-r-- 1 ye ye 3.4M Jan 29 00:25 ./model/name/gpt.ftfz.model
-rw-rw-r-- 1 ye ye 3.7M Jan 29 00:21 ./model/name/mlp.ftfz.model
-rw-rw-r-- 1 ye ye 3.4M Jan 29 00:24 ./model/name/transformer.ftfz.model


In [8]:
!wc ./model/name/*ftfz*vocab

  1443   2886  24556 ./model/name/bert.ftfz.model.vocab
  1443   2886  24556 ./model/name/bilstm.ftfz.model.vocab
  1443   2886  24556 ./model/name/gpt.ftfz.model.vocab
  1443   2886  24556 ./model/name/mlp.ftfz.model.vocab
  1443   2886  24556 ./model/name/transformer.ftfz.model.vocab
  7215  14430 122780 total


In [18]:
!head ./model/name/gpt.ftfz.model.vocab

ပယ်	0
လို့	1
ချဲ့	2
အိန္မာ	3
ကျူး	4
သည်း	5
ရှိတ်	6
ရွိုင်	7
သွယ်	8
နယ်	9


In [20]:
!head ./model/name/transformer.ftfz.model.vocab

လင်္ကာ	0
ကျို	1
ဇေ	2
ယန်း	3
ဒိန်	4
စုံ	5
မတ်	6
ဂျေ	7
ဇိန်း	8
တည်	9


မော်ဒယ် တစ်ခုချင်းစီရဲ့ generated name တွေကို လေ့လာကြည့်ရအောင်။   

In [21]:
!cat ./output/name/mlp_ftfz_gen_texts.txt

ကျော် လန်း မွှန်း
ကျော် ဠု ထား
ကျော် ဝန် တာ
ကျော် ကျ ကြွမ်
ကျော် ကင်း မြင့့််
မ မ တန် မင်္ဂ
မ မ ချင်း ဿီ
မ မ ဥမ္မာ ဋာ
မ မ ကျီး ဟယ်
မ မ ယော ခွေ
အေး ဂို လှဲ
အေး ဆီ ဟွ
အေး မြန် ခု
အေး လျှား သိမ်း
အေး ညွန် ရေး
လှ လှ ဇက် ခါ
လှ လှ ရှောင် စဲ
လှ လှ ကွမ်း အိမ်း
လှ လှ ချက်ထ် နေ
လှ လှ ဆုန်း နွေ
စွန် ချက်ထ် လိ
စွန် ဆာ ရစ်
စွန် လွေ ရွန်း
စွန် ဒေါင်း သင့်
စွန် အဲန်း ကြ
မြ အေး ဟ ခေါင်
မြ အေး အိန္တာ ဠု
မြ အေး သန်း ပြောင်း
မြ အေး ဝံ ဠု
မြ အေး လမ် ဟောမ်း
သ နိုက် ဿဏ်
သ အဲ ပွန်း
သ ဟေ သု
သ ခမ်း မွမ်
သ ရိုး ဆိုး
မောင် နာ့ သောင်
မောင် ကွဲ အော်
မောင် သား ခြူး
မောင် ဂျမ် ခြူး
မောင် ဇမ်း ထိန်
မြင့် မြင့် အိင်္ခ ဗို
မြင့် မြင့် လင်း ထွတ်
မြင့် မြင့် အဲ ကမ်
မြင့် မြင့် ဟိန်း ပယ်
မြင့် မြင့် [PAD] တီးလ်
ရွှေ ဒတ် မွေး
ရွှေ အုံ ပွန်
ရွှေ အု ပွား
ရွှေ မူလ္လာ ရုန်း
ရွှေ ဝီ ရုံ
အဂ္ဂ သျှမ် ခြာ
အဂ္ဂ ရှင်း နှိုင်း
အဂ္ဂ လမ်း သော
အဂ္ဂ ပဉ္စ နောင့်
အဂ္ဂ ရီး ဗြာ
ဥက္ကာ မြေ့ သွင့်
ဥက္ကာ စိ ရိုင်းန်
ဥက္ကာ ထီ ခါရ်
ဥက္ကာ လျန် ထပ်
ဥက္ကာ ရော ထ
သိင်္ဂီ ခမ်း ဇင်
သိင်္ဂီ လဲန်း နေး
သိင်္ဂီ မြဲ ကြည်
သိင်္ဂီ ညို့ လျှန်
သိင်္ဂီ ရုံး ဂန်
မေ 

In [22]:
!cat ./output/name/bilstm_ftfz_gen_texts.txt

ကျော် ဿန်း ဟူ
ကျော် နှီး ဿန်း
ကျော် ရှဲလ် ဣန္တာ
ကျော် ဣန္တာ မျာ
ကျော် ဟူ ရှဲလ်
မ မ ဿန်း နှီး
မ မ ဟူ နှီး
မ မ ရှဲလ် ဣန္တာ
မ မ ဣန္တာ ရှဲလ်
မ မ ရှဲလ် ဿန်း
အေး ရှဲလ် ဌေး
အေး မျာ ဿန်း
အေး ဟူ ဣန္တာ
အေး ရှဲလ် ဌေး
အေး ရှဲလ် ရှဲလ်
လှ လှ မျာ တန်
လှ လှ မျာ ရှဲလ်
လှ လှ မျာ ဿန်း
လှ လှ မျာ ရှဲလ်
လှ လှ ရှဲလ် မျာ
ထော် ဟူ ရှဲလ်
ထော် မျာ ရှဲလ်
ထော် ဌေး ဌေး
ထော် ရှဲလ် မျာ
ထော် ရှဲလ် ဟူ
မြ အေး ရှဲလ် နှီး
မြ အေး တန် တန်
မြ အေး မျာ ဟူ
မြ အေး နှီး နှီး
မြ အေး ဟူ နှီး
သ မျာ မျာ
သ ဌေး နှီး
သ မျာ ဿန်း
သ ဿန်း မျာ
သ ရှဲလ် ဿန်း
မောင် ဟူ နှီး
မောင် ဣန္တာ ရှဲလ်
မောင် တန် နှီး
မောင် ဣန္တာ နှီး
မောင် ဿန်း တန်
မြင့် မြင့် ရှဲလ် ရှဲလ်
မြင့် မြင့် ဣန္တာ တန်
မြင့် မြင့် တန် ရှဲလ်
မြင့် မြင့် မျာ ဌေး
မြင့် မြင့် ဿန်း ဿန်း
ရွှေ မျာ ရှဲလ်
ရွှေ ဣန္တာ ဿန်း
ရွှေ တန် ဣန္တာ
ရွှေ ဌေး ဿန်း
ရွှေ မျာ ဟူ
အဂ္ဂ ဣန္တာ နှီး
အဂ္ဂ ရှဲလ် နှီး
အဂ္ဂ ဟူ နှီး
အဂ္ဂ ရှဲလ် ဟူ
အဂ္ဂ နှီး တန်
ဥက္ကာ တန် နှီး
ဥက္ကာ မျာ ဌေး
ဥက္ကာ ရှဲလ် နှီး
ဥက္ကာ ဌေး နှီး
ဥက္ကာ မျာ မျာ
သိင်္ဂီ တန် ဌေး
သိင်္ဂီ ဿန်း တန်
သိင်္ဂီ မျာ ဌေး
သိင်္ဂီ ဿန်း နှီး
သိင်္ဂီ နှီး မျာ
မေ

In [23]:
!cat ./output/name/transformer_ftfz_gen_texts.txt

ကျော် ပြိုင် ကြွက်
ကျော် ဖြင့် ခူး
ကျော် အိန္ဒြေ သက်
ကျော် ဇိန်း ကြွက်
ကျော် ကျုံး ခေါက်
မ မ ရွမ် ခူး
မ မ လူ ခယ်
မ မ လှေ လွမ်း
မ မ လာမ် ခွမ်း
မ မ ချိ ချ
အေး ညိမ်း ဖောင်း
အေး ခေါ် သျှန်
အေး လင် ရန်
အေး ကျီ သျှန်
အေး ဂေါ် ရန်
လှ လှ ကယ် ရန်
လှ လှ ချက်ထ် ကြွက်
လှ လှ သျှန္တီ ရန်
လှ လှ လောင်း ရန်
လှ လှ ထွန်း ကိုင်
ကန် သျှန် သျှန်
ကန် ခူး သျှန်
ကန် လွမ်း သျှန်
ကန် သျှန် သျှန်
ကန် သျှန် သျှန်
မြ အေး ဒို့ဗ် သျှန်
မြ အေး ထက် ညို
မြ အေး လင်္ကာ ခူး
မြ အေး ဇ ခေတ်
မြ အေး သိင်္ဃ သျှန်
သ ဒိန် ခူး
သ ကစ် သျှန်
သ ယာန် ကြွက်
သ မီး လောဒ်
သ သွေး ကွိဇ်
မောင် သျှန်း သျှန်
မောင် ဗန်း ခူး
မောင် ဝိုင်း ခူ
မောင် ရှောင် သီ
မောင် ဟင်း ခူး
မြင့် မြင့် ကျိမ်း သျှန်
မြင့် မြင့် သေး သျှန်
မြင့် မြင့် ကျီ ခူး
မြင့် မြင့် တုံ သျှန်
မြင့် မြင့် စီး ရန်
ရွှေ မှု ရန်
ရွှေ ထင် ညာ
ရွှေ ရွက် သျှန်
ရွှေ ကျုံ သျှန်
ရွှေ စိမ် သျှန်
အဂ္ဂ သျှန် သျှန်
အဂ္ဂ သျှန် သျှန်
အဂ္ဂ သျှန် သျှန်
အဂ္ဂ ခူး သျှန်
အဂ္ဂ သျှန် သျှန်
ဥက္ကာ ဂျယ်လ် သျှန်
ဥက္ကာ လက် ဟောက်
ဥက္ကာ နော ခူး
ဥက္ကာ ဣန္တာ သျှန်
ဥက္ကာ ခံ သျှန်
သိင်္ဂီ အိုင် သျှန်
သိင်္ဂီ ထင် အဲလ်

In [24]:
!cat ./output/name/bert_ftfz_gen_texts.txt

ကျော် ယော ပီ
ကျော် ညို နန္ဒ
ကျော် သိမ့် ဇော်
ကျော် ကြိုး လှေ
ကျော် ဇိ ရှား
မ မ လဲ လှေ
မ မ ပျို ရှား
မ မ ဒေ သိဒ္ဓိ
မ မ မြီ ရှား
မ မ ပွင့်် သေ့
အေး တောင်း လှေ
အေး ညှင်း ရှား
အေး ဣန္ဒြေ ကျိမ်း
အေး နာ့ ကျင်း
အေး ကုမ္မာ ဝိုက်
လှ လှ ရွာ ပီ
လှ လှ နွန် ရှား
လှ လှ ချောင်း ပီ
လှ လှ နှင်း သင်္ချာ
လှ လှ ကျီး ရှား
သိင်္ခ ရှား ရ
သိင်္ခ ရှား ဝိုက်
သိင်္ခ ရှား ရှား
သိင်္ခ ရှား ဝိုက်
သိင်္ခ ရှား ရှား
မြ အေး ဘူ ရှား
မြ အေး ဒေ ဘောမ်
မြ အေး ချိတ် ရှား
မြ အေး တုန် ရှား
မြ အေး ရေ ကောက်
သ ကျော့် ဟုမ်
သ ယွန်း မြူ
သ ဆွေ ဟင်
သ နို သိဒ္ဓိ
သ ယိမ့် ကျိမ်း
မောင် သိင်္ဂီ ချွန်
မောင် လာန်း လှေ
မောင် ရော့ခ် ပီ
မောင် နှဲမ် နှင်း
မောင် ဖု ကျိမ်း
မြင့် မြင့် ကောန် ရှား
မြင့် မြင့် ဂွမ် လှေ
မြင့် မြင့် နန္ဒ ပီး
မြင့် မြင့် သင်္ချာ ကျိမ်း
မြင့် မြင့် ဝိုင်း ဒမ်း
ရွှေ ဓမ္မာ ပီ
ရွှေ ဿာ ရှား
ရွှေ ဟေ ခွမ်း
ရွှေ ဂိုး ရှား
ရွှေ ထွေ ပီ
အဂ္ဂ ပီ သိမ်း
အဂ္ဂ ရှား ရှား
အဂ္ဂ ပီ သင်္ကြန်
အဂ္ဂ ရ မှီ
အဂ္ဂ ပီ အိမ့်
ဥက္ကာ ဿန် လှေ
ဥက္ကာ လှဲန် နှင်း
ဥက္ကာ ရေး ရှား
ဥက္ကာ အေ လွန်
ဥက္ကာ ကြင် အဲမ်
သိင်္ဂီ ဘွဲ ရှား
သိင်္ဂီ ဗေ ရှား
သိင်္ဂီ ဟြေ ဝိုက

In [25]:
!cat ./output/name/gpt_ftfz_gen_texts.txt

ကျော် ခေါက် ဗေ
ကျော် ဂို ရွတ်
ကျော် သမ္ဘူ ရ
ကျော် ပိုင် ဝီ
ကျော် ဝင့် ရယ်လ်
မ မ ရိုင်း သီ
မ မ မောရ် ဣ
မ မ အိုမ် မြင်
မ မ ဒေါင်း အဉ္ဇူ
မ မ ဆိုင် အိန်
အေး ခွာ ကေ
အေး ခုံ လဲန်း
အေး ပဲန် ဗ
အေး လာန် သန္တာ
အေး နေ မြင့်
လှ လှ ထာ ဘင်း
လှ လှ ငြိမ့် မင်္ဂ
လှ လှ ကီး ဂန္ဓ
လှ လှ လွှင် ကြွမ်
လှ လှ ဇွန် ဒီ
ဂျယ်လ် တေ အေ
ဂျယ်လ် အိုက် ကျောက်
ဂျယ်လ် ကျင် အူလ္လာ
ဂျယ်လ် အဲန် ရစ္စ
ဂျယ်လ် ထွေ ခွား
မြ အေး အဲန် ခင်
မြ အေး ရို မှုန်
မြ အေး စမ်း လှောင်
မြ အေး ထွန်း လွဲ
မြ အေး စစ် လာဘ်
သ ဖ ဒေ့ဗ်
သ တွန်း သန္ဒြာ
သ ချစ် ဂန္တ
သ မြတ် ဒူး
သ ရ ရှာ
မောင် ဟွမ်း ဇမ့်
မောင် ဇာ တန့်
မောင် ခတ် ထာသ်
မောင် ရက် အိန္တာ
မောင် ဟ ဆ
မြင့် မြင့် ဆား ချင်
မြင့် မြင့် လိန်း ရင်န်
မြင့် မြင့် ပစ် တန့်
မြင့် မြင့် ဟူး အံ
မြင့် မြင့် ကန်း မြန်
ရွှေ ဘူး ခု
ရွှေ ဒြာ နီ
ရွှေ ကွမ်း တော
ရွှေ သု ဝိ
ရွှေ ကြော အိုး
အဂ္ဂ တောင်း ပိုး
အဂ္ဂ ကတ် ညို
အဂ္ဂ လှန် တက္ခ
အဂ္ဂ ဒင့် ယူ
အဂ္ဂ ညွတ် သန်း
ဥက္ကာ ငြိမ်း ဂျိုး
ဥက္ကာ လမ်း စန္ဒီ
ဥက္ကာ ကြောင်း လိန်
ဥက္ကာ ဿီ တေး
ဥက္ကာ သဲ ကြည်
သိင်္ဂီ ဇေါင်း မီး
သိင်္ဂီ ဆွိ ဟွေ
သိင်္ဂီ ကီလ် ဇမ်း
သိင်္ဂီ ဂျတ် ဂျယ်လ်
သိင်္ဂီ 

Testing data နဲ့ evaluation လုပ်ထားတဲ့ perplexity, cross-entropy ရလဒ်တွေကို လေ့လာကြည့်ရအောင်။  

In [26]:
!tail -n 2 ./log/name/*ftfz*

==> ./log/name/bert.ftfz.log <==
Average Perplexity on Test Data: 1.0112
Average Cross-Entropy on Test Data: 0.0111

==> ./log/name/bilstm.ftfz.log <==
Average Perplexity on Test Data: 1.1671
Average Cross-Entropy on Test Data: 0.1544

==> ./log/name/gpt.ftfz.log <==
Average Perplexity on Test Data: 1.0015
Average Cross-Entropy on Test Data: 0.0015

==> ./log/name/mlp.ftfz.log <==
Average Perplexity on Test Data: 1.1143
Average Cross-Entropy on Test Data: 0.1082

==> ./log/name/transformer.ftfz.log <==
Average Perplexity on Test Data: 1.0106
Average Cross-Entropy on Test Data: 0.0105
